In [53]:
import os
import csv
import nltk
import math
import shlex
import itertools
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from string import punctuation
from datetime import datetime
from nltk.util import ngrams
from collections import Counter

from pandas import Series
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer

## Clean tags per df

### Clean function

In [306]:
def clean(s):
    r = s.lower().strip()
    for c in chars:
        r = r.replace(c, ' ')
    rs = [w for w in word_tokenize(r) if w not in stop and len(w)>2]
    rs = [lemmatizer.lemmatize(w) for w in rs]
#     r = ' '.join(rs)
#     return list(set(rs))
    return rs

## Define variables/folders/filenames

In [257]:
data_folder_channels='/media/aruiz/data/videos_data/'
data_folder='/media/aruiz/data/videos_tags/'
filename=os.path.join(data_folder,'tags_videosPerChannel4.csv')
fieldnames = ['year', 'channelId','tags','channelTitle', 'previous_community']
output=open(filename, 'a')
writer = csv.DictWriter(output, fieldnames=fieldnames)
# writer.writeheader()

community_num=[9,22,85,76,71,37,87]
lemmatizer = WordNetLemmatizer()
stop= set(stopwords.words('english'))
# exclude = set(punctuation)

## Save all info in a csv

In [258]:
for ind in range (0,len(community_num)):
    
    # create csv to read community video tags
    file3=os.path.join(data_folder_channels,'data_community_'+str(community_num[ind])+'.csv')
    # put data into a dataframe
    df=pd.read_csv(file3)
    df=df.fillna("N/A")
    df=df[~(df['id']=='id')]
    df=df.reset_index(drop=True)
    
    df['year']= df.publishedAt.apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ').year)
    
    # set chars per df
    chars = set()
    for tag in df.tags:
        for ch in tag:
            if not ch.isalpha():
                chars.add(ch)
                    
    df['clean_tags']= df.tags.apply(lambda x: clean(x))
    # choose only the tags repeated more than few times

    df2=df.groupby(['year','channelId']).clean_tags.apply(lambda x: x.sum())
    
    df.set_index('channelId', inplace=True)
#     df3.drop_duplicates(subset=['channelTitle'], keep='first', inplace= True)
    
    for params,tags in df2.items():
        dc={}
        counts= Series(tags)
        cou=Counter(counts).most_common(20)
        all_tags=''
        for tup in cou:
            all_tags+=tup[0]+' '
            
        dc['year']=params[0]
        dc['channelId']=params[1]
        dc['tags']=all_tags
        dc['channelTitle']=df.at[params[1], 'channelTitle'][0]
        dc['previous_community']=community_num[ind]
        #write each dictionary in a csv file
        writer.writerow(dc)
        
    print('finished community: '+ str(community_num[ind]))
    
output.close()

/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,9,10,11,12,13,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


finished community: 9


/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,9,10,11,12,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


finished community: 22
finished community: 85
finished community: 76
finished community: 71
finished community: 37
finished community: 87


## Read csv and create a network by year

In [161]:
data_folder='/media/aruiz/data/videos_tags/'
file=os.path.join(data_folder,'tags_videosPerChannel4.csv')
df=pd.read_csv(file)

In [174]:
for y in range (2016,2017):
    df2=df[df['year']==y]
    df2.reset_index(drop=True, inplace=True)
    df2['temp']=df2.tags.apply(lambda x: set(str(x).split()))
    break

/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


## Adding nodes

In [187]:
g=nx.Graph()
for i in range (0, len(df2)):
    g.add_node(df2.channelId[i], Label=df2.channelTitle[i],p_community=int(df2.previous_community[i]))

## Adding edges with weight calculation

In [188]:
# fanp=['mod', 'machinima', 'montage', 'let', 'walkthrough', 'tutorial']
fanp=['game', 'video', 'call', 'duty', 'minecraft', 'grand', 'theft', 'auto',
      'agar.io','overwatch','roblox','five', 'nights', 'freddys','pokemon', 'go', 'happy', 'wheels', 'league', 'legends',
      'gameplay', 'gaming']

In [189]:
for i in range (0, len(df2)-1):
    for j in range (i+1,len(df2)):
        shared_items=df2.temp[i]&df2.temp[j]
        fanTags=[s for s in shared_items if s not in fanp]
#         shared_len=len(shared_items)
        shared_len=len(fanTags)
        if shared_len>2:
            g.add_edge(df2.channelId[i],df2.channelId[j], weight=int(shared_len), fanTag=' '.join(fanTags))

## Save network in file

In [181]:
print(len(g.nodes()))
print(len(g.edges()))

1819
165545


In [182]:
h=g.copy()
a=nx.isolates (g)
h.remove_nodes_from(a)

In [183]:
print(len(h.nodes()))
print(len(h.edges()))

1721
165545


In [185]:
nx.write_gexf(h,"similarity_videos_ch2016_fanp2.gexf")

In [184]:
shared_list=[]
for n1,n2,data in h.edges(data=True):
    shared_list+=(data['fanTag'].split())
most_shared= Counter(Series(shared_list)).most_common()[0:30]
print(most_shared)

[('gameplay', 90883), ('play', 48867), ('black', 38152), ('let', 36484), ('ops', 34715), ('funny', 34341), ('cod', 23292), ('gaming', 19651), ('moment', 16373), ('walkthrough', 14695), ('new', 14297), ('xbox', 13934), ('best', 13525), ('gta', 12087), ('warfare', 11159), ('online', 10982), ('one', 9999), ('playthrough', 9858), ('playstation', 7921), ('mod', 7480), ('zombie', 7466), ('multiplayer', 7338), ('live', 6593), ('dlc', 5805), ('montage', 5466), ('sony', 5452), ('entertainment', 4888), ('glitch', 4569), ('commentary', 4560), ('guide', 3687)]


## Count other kinds of fan production 

In [66]:
def clean_unique(s):
    r = s.lower().strip()
    for c in chars:
        r = r.replace(c, ' ')
    rs = [w for w in word_tokenize(r) if w not in stop and len(w)>2]
    rs = [lemmatizer.lemmatize(w) for w in rs]
    r = ' '.join(list(set(rs)))
#     return list(set(rs))
    return r+' '

In [74]:
data_folder_channels='/media/aruiz/data/videos_data/'
data_folder='/media/aruiz/data/videos_tags/'
filename=os.path.join(data_folder,'frequency_tags_community_final_lk.csv')
fieldnames = ['channelId','year', 'previous_community','counts']
output=open(filename, 'a')
writer = csv.DictWriter(output, fieldnames=fieldnames)
# writer.writeheader()

community_num=[9,22,85,76,71,37,87]
lemmatizer = WordNetLemmatizer()
stop= set(stopwords.words('english'))
# exclude = set(punctuation)

In [50]:
for ind in range (0,len(community_num)):
    # create csv to read community video tags
    file2=os.path.join(data_folder_channels,'data_community_'+str(community_num[ind])+'.csv')
    # put data into a dataframe
    df=pd.read_csv(file2)
    df=df[~(df['id']=='id')]
    #------------------------------------------
    df=df.fillna(" ")
    if ind==3 or ind==5 or ind==6:
        df=df[df['categoryId']=='20']
    else:
        df=df[df['categoryId']==20]
    #------------------------------------------
    df=df.reset_index(drop=True)
    
    df['year']= df.publishedAt.apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ').year)
    
    # set chars per df
    chars = set()
    for tag in df.tags:
        for ch in tag:
            if not ch.isalpha():
                chars.add(ch)
                    
    df['clean_tags']= df.tags.apply(lambda x: clean_unique(x))
    # choose only the tags repeated more than few times
    
    df2=df.groupby(['year','channelId'])
    df3=df.groupby(['year','channelId']).clean_tags.sum()
    
    for params, dic in df2:
        x=len(dic)
        tags=df3[params[0],params[1]]
        count=Counter(Series(tags.split()))
        for c in count:
            count[c]=count[c]*(100/x)
        df3[params[0],params[1]]=count
    
    df4=df3.groupby('year')
    df3=df3.groupby('year').sum()
   
    for year, dic in df4:
        dc={}
        x=len(dic)
        tag_count=df3[year]
        for tag in tag_count:
            tag_count[tag]=tag_count[tag]/x
#         df3[year]=tag_count
        dc['year']=year
        dc['previous_community']=community_num[ind]
        dc['counts']=tag_count
        writer.writerow(dc)
    
    print('finished community: '+ str(community_num[ind]))
    
output.close()

/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,9,10,11,12,13,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


finished community: 9


/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,9,10,11,12,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


finished community: 22
finished community: 85
finished community: 76
finished community: 71
finished community: 37
finished community: 87


In [68]:
for ind in range (0,len(community_num)):
    # create csv to read community video tags
    file2=os.path.join(data_folder_channels,'data_community_'+str(community_num[ind])+'.csv')
    # put data into a dataframe
    df=pd.read_csv(file2)
    df=df[~(df['id']=='id')]
    #------------------------------------------
    df=df.fillna(" ")
    if ind==3 or ind==5 or ind==6:
        df=df[df['categoryId']=='20']
    else:
        df=df[df['categoryId']==20]
    #------------------------------------------
    df=df.reset_index(drop=True)
    
    df['year']= df.publishedAt.apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ').year)
    
    # set chars per df
    chars = set()
    for tag in df.tags:
        for ch in tag:
            if not ch.isalpha():
                chars.add(ch)
                    
    df['clean_tags']= df.tags.apply(lambda x: clean_unique(x))
#     df['clean_titles']=df.tags.apply(lambda x: clean_unique(x))
    # choose only the tags repeated more than few times
    
    df2=df.groupby(['year','channelId'])
    df3=df.groupby(['year','channelId']).clean_tags.sum()
    
    for params, dic in df2:
        x=len(dic)
        tags=df3[params[0],params[1]]
        count=Counter(Series(tags.split()))
#         for c in count:
#             count[c]=count[c]*(100/x)
        df3[params[0],params[1]]=count
    
#     df4=df3.groupby('year')
#     df3=df3.groupby('year').sum()
   
#     for year, dic in df4:
    for params, dic in df3.items():
        dc={}
#         x=len(dic)
#         tag_count=df3[year]
#         for tag in tag_count:
#             tag_count[tag]=tag_count[tag]/x
# #         df3[year]=tag_count
        dc['year']=params[0]
        dc['previous_community']=community_num[ind]
        dc['channelId']=params[1]
        dc['counts']=dic
        writer.writerow(dc)
    
    print('finished community: '+ str(community_num[ind]))
    
output.close()

/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,9,10,11,12,13,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


finished community: 9


/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,9,10,11,12,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


finished community: 22
finished community: 85
finished community: 76
finished community: 71
finished community: 37
finished community: 87


In [311]:
df3=df.groupby(['year','channelId']).clean_tags.sum()

In [252]:
%%time
df['clean_tags']= df.tags.apply(lambda x: clean_unique(x))

In [312]:
for params, dic in df2:
    x=len(dic)
    tags=df3[params[0],params[1]]
    count=Counter(Series(tags.split()))
    for c in count:
        count[c]=round(count[c]*(100/x),1)
    df3[params[0],params[1]]=count

In [51]:
df4=df3.groupby('year')
df3=df3.groupby('year').sum()

In [314]:
for year, dic in df4:
    x=len(dic)
    tag_count=df3[year]
    for tag in tag_count:
        tag_count[tag]=round(tag_count[tag]/x,1)
        
    df3[year]=tag_count

In [52]:
df3[2012].most_common(20)

In [80]:
for ind in range (0,len(community_num)):
    # create csv to read community video tags
    file2=os.path.join(data_folder_channels,'data_community_'+str(community_num[ind])+'.csv')
    # put data into a dataframe
    df=pd.read_csv(file2)
    df=df[~(df['id']=='id')]
    #------------------------------------------
    df=df.fillna(" ")
    if ind==3 or ind==5 or ind==6:
        df=df[df['categoryId']=='20']
    else:
        df=df[df['categoryId']==20]
    #------------------------------------------
    df=df.reset_index(drop=True)
    
    df['year']= df.publishedAt.apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ').year)
    
    # set chars per df
    chars = set()
    for tag in df.tags:
        for ch in tag:
            if not ch.isalpha():
                chars.add(ch)
                    
    df['clean_tags']= df.tags.apply(lambda x: clean_unique(x))
    
    df['previous_community']=community_num[ind]
    
    with open('frequency_tags_complete.csv', 'a') as f:
        df.to_csv(f, index=False)
        
    print('wrote '+ str(community_num[ind]))

/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,9,10,11,12,13,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


wrote 9


/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,9,10,11,12,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


wrote 22
wrote 85
wrote 76
wrote 71
wrote 37
wrote 87


In [81]:
df=pd.read_csv('frequency_tags_complete.csv')

/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,9,11,16,19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [83]:
df.head().clean_tags[0]

'map trolling gaming spotlight build review lanceypooh glitch adventure lancey commentary multiplayer mod forge montage popular play troll video quality minecraft funny custom playthrough showcase let survival moment xbox minecart high '

In [142]:
# community_num =[9,22,85,76,71,37,87]
community_num =[9,71]
exclude=['bqNzbkIHYF8', 'ZTjkUpyBnpA']

for n in range (0, len(community_num)):
    mxview=-1
    idvid=''
    description=''
    title=''

    df3=df[df['previous_community']==community_num[n]]
    df3=df3[df3['year']==2011]
    df3.reset_index(inplace=True, drop=True)

    for i in range (0, len (df3)):
        tag=df3.clean_tags[i].split()
        if 'montage' in tag:
            if df3.viewCount[i]!=' ':
                if df3.id[i] not in exclude:
                    if (float(df3.viewCount[i]))>mxview:
                        mxview=float(df3.viewCount[i])
                        idvid=df3.id[i]

    print('')
    print(community_num[n])
    if idvid!='':
        print(df2.description[idvid])
        print(df2.title[idvid])
        print(df2.likeCount[idvid])
        print(df2.year[idvid])
        print(idvid)
        print(mxview)


9
Leave a like, helps me out a good bit. http://www.youtube.com/subscription_center?add_user=uberhaxornova

Our first play of the new zombie map.

LINKSSSS: 

Twitter:
http://twitter.com/UberHaxorNova

Sp00ners Page:
http://www.youtube.com/user/TheCampingTree

Kootras Channel:
http://www.youtube.com/user/kootra

Minecraft Playlist:
http://www.youtube.com/view_play_list?p=76FE625737BE2A4C

ALL BLACK OPS ZOMBIES:
http://tinyurl.com/27cjnez
Call of Duty: Black Ops - Escalation Call of the Dead Zombies w/Nova, Kootra & Spoon #36
5852.0
2011
07JfzsDwJQE
1864230.0

71


In [137]:
df2.tags['ZTjkUpyBnpA']

'[\'pvz\', \'plants\', \'gargantuar boss\', \'zombies\', \'pvz mod\', \'pvz garden warfare\', \'plants vs. zombies\', \'iulitm\', \'fan mods\', \'plants vs. zombies garden warfare\', \'IULITM\', \'FNAC gameplay\', \'torture\', \'Animatronics Jumpscares\', "FNAF Let\'s Play"]'